In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,20.91,37,97,2.51,RU,1690179865
1,1,hawaiian paradise park,19.5933,-154.9731,23.73,97,100,0.00,US,1690179866
2,2,puerto natales,-51.7236,-72.4875,-6.24,94,55,1.24,CL,1690179663
3,3,waitangi,-43.9535,-176.5597,11.01,97,100,7.22,NZ,1690179866
4,4,remire-montjoly,4.9167,-52.2667,25.02,94,0,1.03,GF,1690179866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
cities_map=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=1,
    color="City"
)

cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

filtered_city_data=city_data_df.loc[city_data_df["Cloudiness"]==0]
filtered_city_data.count()

# Drop any rows with null values
filtered_city_data.dropna(how="any")

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,remire-montjoly,4.9167,-52.2667,25.02,94,0,1.03,GF,1690179866
7,7,port elizabeth,-33.9180,25.5701,12.16,66,0,5.14,ZA,1690179867
24,24,tukrah,32.5341,20.5791,30.84,56,0,2.56,LY,1690179870
28,28,i-n-salah,27.1935,2.4607,34.20,10,0,4.52,DZ,1690179871
38,38,rakovski,42.3000,24.9667,23.92,59,0,1.29,BG,1690179873
...,...,...,...,...,...,...,...,...,...,...
543,543,al burayqah,30.4062,19.5739,28.73,70,0,4.11,LY,1690179986
548,548,tamanrasset,22.7850,5.5228,27.95,15,0,5.14,DZ,1690179987
552,552,plymouth,41.8334,-70.8328,22.03,85,0,0.00,US,1690179988
554,554,la ronge,55.1001,-105.2842,15.14,72,0,1.03,CA,1690179988


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
hotel_df=filtered_city_data.copy()
hotel_df=pd.DataFrame(hotel_df.loc[:,["City","Country","Lat","Lng","Humidity",]])
hotel_df["Hotel Name"]=""
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
4,remire-montjoly,GF,4.9167,-52.2667,94,
7,port elizabeth,ZA,-33.9180,25.5701,66,
24,tukrah,LY,32.5341,20.5791,56,
28,i-n-salah,DZ,27.1935,2.4607,10,
38,rakovski,BG,42.3000,24.9667,59,
...,...,...,...,...,...,...
543,al burayqah,LY,30.4062,19.5739,70,
548,tamanrasset,DZ,22.7850,5.5228,15,
552,plymouth,US,41.8334,-70.8328,85,
554,la ronge,CA,55.1001,-105.2842,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
port elizabeth - nearest hotel: Waterford Hotel
tukrah - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
rakovski - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
bethel - nearest hotel: Hampton Inn Danbury
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
mar del plata - nearest hotel: Nuevo Ostende
saint-pierre - nearest hotel: Tropic Hotel
leczna - nearest hotel: No hotel found
dawesville - nearest hotel: Baymont by Wyndham Thomasville
fruita - nearest hotel: Balanced Rock Inn
hamilton - nearest hotel: North Vista Manor
kodiak - nearest hotel: Shelikof Lodge
badger - nearest hotel: Hotel North Pole
sparti - nearest hotel: Maniatis
el ksour - nearest hotel: No hotel found
nova olimpia - nearest hotel: Olímpia Hotel
mersin - nearest hotel: Gondol Oteli
yuma - nearest hotel: Hilton Garden Inn Yuma Pivot Point
east london - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
4,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
7,port elizabeth,ZA,-33.9180,25.5701,66,Waterford Hotel
24,tukrah,LY,32.5341,20.5791,56,No hotel found
28,i-n-salah,DZ,27.1935,2.4607,10,باجودة
38,rakovski,BG,42.3000,24.9667,59,No hotel found
...,...,...,...,...,...,...
543,al burayqah,LY,30.4062,19.5739,70,فندق ماس ليبيا
548,tamanrasset,DZ,22.7850,5.5228,15,فندق الأمان
552,plymouth,US,41.8334,-70.8328,85,Rodeway Inn Middleboro-Plymouth
554,la ronge,CA,55.1001,-105.2842,72,La Ronge Hotel and Suites


In [20]:
# Configure the map plot
hotel_map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["Hotel Name","Country"]
)

# YOUR CODE HERE
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)